# Census Data Wrangle

In this section, I will be cleaning data from 'acs2017_census_tract_data.csv'.

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

%matplotlib inline

In [15]:
# Any needed functions

# returns mean absolute error for random forests.
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=1)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

## Gather

(1) Read CSV file into dataframe

In [3]:
# Read CSV file into dataframe
df = pd.read_csv('acs2017_census_tract_data.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74001 entries, 0 to 74000
Data columns (total 37 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TractId           74001 non-null  int64  
 1   State             74001 non-null  object 
 2   County            74001 non-null  object 
 3   TotalPop          74001 non-null  int64  
 4   Men               74001 non-null  int64  
 5   Women             74001 non-null  int64  
 6   Hispanic          73305 non-null  float64
 7   White             73305 non-null  float64
 8   Black             73305 non-null  float64
 9   Native            73305 non-null  float64
 10  Asian             73305 non-null  float64
 11  Pacific           73305 non-null  float64
 12  VotingAgeCitizen  74001 non-null  int64  
 13  Income            72885 non-null  float64
 14  IncomeErr         72885 non-null  float64
 15  IncomePerCap      73256 non-null  float64
 16  IncomePerCapErr   73256 non-null  float6

In [5]:
df.head()

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001020100,Alabama,Autauga County,1845,899,946,2.4,86.3,5.2,0.0,1.2,0.0,1407,67826.0,14560.0,33018.0,6294.0,10.7,20.8,38.5,15.6,22.8,10.8,12.4,94.2,3.3,0.0,0.5,0.0,2.1,24.5,881,74.2,21.2,4.5,0.0,4.6
1,1001020200,Alabama,Autauga County,2172,1167,1005,1.1,41.6,54.5,0.0,1.0,0.0,1652,41287.0,3819.0,18996.0,2453.0,22.4,35.8,30.5,24.9,22.9,6.3,15.4,90.5,9.1,0.0,0.0,0.5,0.0,22.2,852,75.9,15.0,9.0,0.0,3.4
2,1001020300,Alabama,Autauga County,3385,1533,1852,8.0,61.4,26.5,0.6,0.7,0.4,2480,46806.0,9496.0,21236.0,2562.0,14.7,21.1,27.9,19.4,33.3,9.9,9.6,88.3,8.4,0.0,1.0,0.8,1.5,23.1,1482,73.3,21.1,4.8,0.7,4.7
3,1001020400,Alabama,Autauga County,4267,2001,2266,9.6,80.3,7.1,0.5,0.2,0.0,3257,55895.0,4369.0,28068.0,3190.0,2.3,1.7,29.0,16.6,25.8,9.1,19.5,82.3,11.2,0.0,1.5,2.9,2.1,25.9,1849,75.8,19.7,4.5,0.0,6.1
4,1001020500,Alabama,Autauga County,9965,5054,4911,0.9,77.5,16.4,0.0,3.1,0.0,7229,68143.0,14424.0,36905.0,10706.0,12.2,17.9,48.8,13.8,20.5,3.5,13.4,86.9,11.2,0.0,0.8,0.3,0.7,21.0,4787,71.4,24.1,4.5,0.0,2.3


## Assess


### Quality

* (1) There are 158 rows with null values in only one column.

#### Assessment 1

In [6]:
#Create list of index values for rows that contain null values in only one column.
null_count = list(df[df.isnull().sum(axis = 1) == 1].index.values.tolist())

In [7]:
len(null_count)

158

### Tidiness

* (2) The data in the 'County' column contains the word 'County' in all rows.

#### Assessment 2

In [8]:
df.head()

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001020100,Alabama,Autauga County,1845,899,946,2.4,86.3,5.2,0.0,1.2,0.0,1407,67826.0,14560.0,33018.0,6294.0,10.7,20.8,38.5,15.6,22.8,10.8,12.4,94.2,3.3,0.0,0.5,0.0,2.1,24.5,881,74.2,21.2,4.5,0.0,4.6
1,1001020200,Alabama,Autauga County,2172,1167,1005,1.1,41.6,54.5,0.0,1.0,0.0,1652,41287.0,3819.0,18996.0,2453.0,22.4,35.8,30.5,24.9,22.9,6.3,15.4,90.5,9.1,0.0,0.0,0.5,0.0,22.2,852,75.9,15.0,9.0,0.0,3.4
2,1001020300,Alabama,Autauga County,3385,1533,1852,8.0,61.4,26.5,0.6,0.7,0.4,2480,46806.0,9496.0,21236.0,2562.0,14.7,21.1,27.9,19.4,33.3,9.9,9.6,88.3,8.4,0.0,1.0,0.8,1.5,23.1,1482,73.3,21.1,4.8,0.7,4.7
3,1001020400,Alabama,Autauga County,4267,2001,2266,9.6,80.3,7.1,0.5,0.2,0.0,3257,55895.0,4369.0,28068.0,3190.0,2.3,1.7,29.0,16.6,25.8,9.1,19.5,82.3,11.2,0.0,1.5,2.9,2.1,25.9,1849,75.8,19.7,4.5,0.0,6.1
4,1001020500,Alabama,Autauga County,9965,5054,4911,0.9,77.5,16.4,0.0,3.1,0.0,7229,68143.0,14424.0,36905.0,10706.0,12.2,17.9,48.8,13.8,20.5,3.5,13.4,86.9,11.2,0.0,0.8,0.3,0.7,21.0,4787,71.4,24.1,4.5,0.0,2.3


## Clean

#### Assessment 1 - Define

I will use a random forest regressor to predict the values of the 133 null values in 'ChildPoverty', while applying the mean to the 25 in 'MeanCommute'.

#### Assessment 1 - Code

In [9]:
# Create seperate dataframe that removes all rows will null values for the purpose of ML training.
df_pure = df.dropna()

In [10]:
# Create seperate dataframe that contains only the targeted rows, along with X and y to apply the model.
df_missing = df.loc[null_count]
df_missing = df_missing.query('ChildPoverty != ChildPoverty')
m_y = df_missing.ChildPoverty
m_features = list(df_missing.columns)
m_features = [c for c in m_features if c not in ('TractId', 'State', 'County', 'TotalPop', 'ChildPoverty')]
m_X = df_missing[m_features]

In [11]:
# Create X and y for training the model.
y = df_pure.ChildPoverty
features = list(df_pure.columns)
features = [c for c in features if c not in ('TractId', 'State', 'County', 'TotalPop', 'ChildPoverty')]
X = df_pure[features]

In [12]:
# Split into training and validation data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1)

In [13]:
# Specify and fit model.
c_poverty_model = RandomForestRegressor(random_state = 1)
c_poverty_model.fit(train_X, train_y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)

In [16]:
#Write loop to find the ideal tree size. 
candidate_max_leaf_nodes = [5, 15, 25, 50, 100, 200, 500, 700, 1000]
for max_leaf_nodes in candidate_max_leaf_nodes:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print('Max leaf Nodes: %d \t\t Mean absolute error: %d' %(max_leaf_nodes, my_mae))

Max leaf Nodes: 5 		 Mean absolute error: 6
Max leaf Nodes: 15 		 Mean absolute error: 5
Max leaf Nodes: 25 		 Mean absolute error: 5
Max leaf Nodes: 50 		 Mean absolute error: 4
Max leaf Nodes: 100 		 Mean absolute error: 4
Max leaf Nodes: 200 		 Mean absolute error: 4
Max leaf Nodes: 500 		 Mean absolute error: 4
Max leaf Nodes: 700 		 Mean absolute error: 4
Max leaf Nodes: 1000 		 Mean absolute error: 4


In [17]:
# Save best tree size
best_tree_size = 100

In [18]:
#Apply model to the validation data and calculate mae.
c_poverty_model = RandomForestRegressor(max_leaf_nodes = best_tree_size, random_state = 1)
c_poverty_model.fit(train_X, train_y)
val_predictions = c_poverty_model.predict(val_X)
val_mae = mean_absolute_error(val_predictions, val_y)
print('Val error for best_tree_size: {:,.0f}'.format(val_mae))

Val error for best_tree_size: 5


In [19]:
# Specify and fit final model and apply to new data.
final_model = RandomForestRegressor(max_leaf_nodes = best_tree_size)
final_model.fit(X, y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=100,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [20]:
# Create list of predictions for missing values
c_poverty_predict = final_model.predict(m_X)

In [21]:
# round list to 1 decimal place.
c_poverty_predict = [round(num, 1) for num in c_poverty_predict]

In [22]:
# Create list of tract ids for all target rows.
id_list = df_missing['TractId']

In [23]:
# Convert to list
id_list = list(id_list)

In [24]:
print(len(id_list), len(c_poverty_predict))

133 133


In [25]:
# Create function to apply values to full df.
list_position = 0
c_poverty_list = []
def apply_values(row):
    global list_position
    if row['TractId'] == id_list[list_position]:
        c_poverty_list.append(c_poverty_predict[list_position])
        if list_position != len(id_list) - 1:
            list_position += 1
    else:
        c_poverty_list.append(row['ChildPoverty'])    

df.apply(apply_values, axis = 1)
df['ChildPoverty'] = c_poverty_list

#### Test

In [26]:
# Look at the 'ChildPoverty' column of the target rows.
df[df['TractId'].isin(id_list)]

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
1124,1125011200,Alabama,Tuscaloosa County,7307,3454,3853,2.1,82.1,10.2,0.3,2.9,0.3,7004,25833.0,5071.0,5632.0,608.0,16.7,21.4,33.6,30.4,33.7,1.0,1.3,42.2,1.8,1.9,22.5,6.5,25.0,12.5,1148,47.0,51.0,2.0,0.0,31.2
1466,4013040506,Arizona,Maricopa County,5404,2401,3003,0.3,97.9,0.5,0.0,0.0,0.0,5226,40978.0,7396.0,33293.0,4986.0,8.2,9.3,41.4,16.1,35.0,2.8,4.7,78.7,3.5,0.0,1.9,4.1,11.9,23.2,859,80.8,4.8,14.4,0.0,14.8
1467,4013040507,Arizona,Maricopa County,6395,2859,3536,0.5,96.7,1.1,0.5,1.0,0.0,6335,42378.0,5270.0,39161.0,7085.0,7.8,8.4,35.8,21.1,31.1,0.0,12.1,83.3,6.2,0.0,0.0,4.4,6.2,25.7,679,84.2,9.1,6.6,0.0,4.9
1468,4013040512,Arizona,Maricopa County,1747,775,972,1.0,95.2,0.9,0.6,0.8,0.0,1666,47271.0,2950.0,35678.0,3843.0,6.0,15.2,17.2,14.7,51.5,6.7,9.8,78.1,4.6,0.0,0.0,10.6,6.6,25.8,163,77.3,12.9,9.8,0.0,6.9
1478,4013040522,Arizona,Maricopa County,4272,1807,2465,0.0,97.8,0.7,0.0,1.1,0.0,4125,46489.0,5565.0,39370.0,7008.0,7.8,9.1,26.4,23.4,45.2,0.0,5.0,62.8,12.1,0.0,0.0,11.3,13.8,31.2,239,79.1,0.0,20.9,0.0,0.0
1480,4013040524,Arizona,Maricopa County,1450,701,749,2.1,93.0,4.4,0.0,0.0,0.0,1392,70444.0,9982.0,41824.0,3370.0,4.2,4.4,51.6,19.7,22.1,1.9,4.7,77.4,0.0,0.0,0.0,5.4,17.2,34.4,213,83.1,7.0,9.9,0.0,9.4
1481,4013040525,Arizona,Maricopa County,3420,1681,1739,2.9,94.5,1.8,0.0,0.6,0.0,3317,68183.0,4197.0,42485.0,3065.0,4.7,4.5,11.3,23.2,46.7,4.1,14.6,64.8,2.5,2.3,0.0,4.8,25.6,28.1,362,68.0,6.9,25.1,0.0,9.3
1482,4013040526,Arizona,Maricopa County,2053,969,1084,3.8,93.1,1.2,0.0,0.9,0.0,1989,57415.0,3467.0,41933.0,6715.0,7.7,39.3,29.8,7.6,51.3,7.6,3.8,43.2,19.7,0.0,0.0,7.4,29.7,21.1,238,96.2,0.0,3.8,0.0,8.5
1484,4013040528,Arizona,Maricopa County,6121,2693,3428,1.5,97.1,0.0,0.3,0.3,0.0,5959,58967.0,6331.0,42917.0,4231.0,4.1,4.0,60.1,9.2,23.5,3.6,3.6,43.9,11.3,0.0,9.0,2.4,33.4,24.2,695,91.2,5.0,3.7,0.0,1.3
1485,4013040529,Arizona,Maricopa County,3046,1398,1648,2.0,95.6,0.0,0.0,2.1,0.0,2982,55104.0,11108.0,41892.0,3813.0,2.6,1.5,46.6,13.9,36.8,0.0,2.7,73.0,0.0,0.0,0.0,8.6,18.4,27.7,337,70.3,15.1,12.2,2.4,2.6


#### Assessment 2 - Define

I will remove the substring ' County' from the entire 'County' column.

#### Assessment 2 - Code

In [28]:
df['County'] = df['County'].str.replace(' County', "")

#### Assessment 2 - Test

In [32]:
# Show all rows where the 'County' column contains the substring 'County'.
df[df['County'].str.contains('County')]

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
